### In this notebook we perform individual training.
In individual learning each base station has access only to it's private dataset.

In [1]:
import sys
import os

from pathlib import Path

parent = Path(os.path.abspath("")).resolve().parents[0]
if parent not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
import random

import numpy as np
import torch

from argparse import Namespace

In [3]:
from ml.utils.data_utils_yl import read_data, generate_time_lags, time_to_feature, handle_nans, to_Xy, \
    to_torch_dataset, to_timeseries_rep, assign_statistics, \
    to_train_val, scale_features, get_data_by_area, remove_identifiers, get_exogenous_data_by_area, handle_outliers

In [4]:
from ml.utils.train_utils import train, test

In [5]:
from ml.models.mlp import MLP
from ml.models.rnn import RNN
from ml.models.lstm import LSTM
from ml.models.gru import GRU
from ml.models.cnn import CNN
from ml.models.rnn_autoencoder import DualAttentionAutoEncoder
args = Namespace(
    data_path='../dataset/full_dataset.csv', # dataset
    data_path_test=['../dataset/ElBorn_test.csv'], # test dataset
    test_size=0.2, # validation size 
    targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'], # the target columns
    num_lags=10, # the number of past observations to feed as input

    
    filter_bs=None, # whether to use a single bs for training. It will be changed dynamically
    identifier='District', # the column name that identifies a bs

    nan_constant=0, # the constant to transform nan values
    x_scaler='minmax', # x_scaler
    y_scaler='minmax', # y_scaler
    outlier_detection=None, # whether to perform flooring and capping

    
    criterion='mse', # optimization criterion, mse or l1
    epochs=150, # the number of maximum epochs
    lr=0.001, # learning rate
    optimizer='adam', # the optimizer, it can be sgd or adam
    batch_size=128, # the batch size to use
    early_stopping=True, # whether to use early stopping
    patience=50, # patience value for the early stopping parameter (if specified)
    max_grad_norm=0.0, # whether to clip grad norm
    reg1=0.0, # l1 regularization
    reg2=0.0, # l2 regularization
    
    plot_history=True, # plot loss history

    cuda=True, # whether to use gpu
    
    seed=0, # reproducibility

    assign_stats=None, # whether to use statistics as exogenous data, ["mean", "median", "std", "variance", "kurtosis", "skew"]
    use_time_features=False # whether to use datetime features
)

> You can define the base station to perform train on the filter_bs parameter and use it in block 12 or you can define the base station to block 12 explicitly 

In [6]:
print(f"Script arguments: {args}\n")

Script arguments: Namespace(assign_stats=None, batch_size=128, criterion='mse', cuda=True, data_path='../dataset/full_dataset.csv', data_path_test=['../dataset/ElBorn_test.csv'], early_stopping=True, epochs=150, filter_bs=None, identifier='District', lr=0.001, max_grad_norm=0.0, nan_constant=0, num_lags=10, optimizer='adam', outlier_detection=None, patience=50, plot_history=True, reg1=0.0, reg2=0.0, seed=0, targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'], test_size=0.2, use_time_features=False, x_scaler='minmax', y_scaler='minmax')



In [7]:
import torch.version


device = "cuda" if args.cuda and torch.cuda.is_available() else "cpu"
print(f"Using {device}")
print("torch_version",torch.__version__)
print("args.cuda",args.cuda)
print("torch.cuda.is_available()",torch.cuda.is_available())

Using cpu
torch_version 2.3.0
args.cuda True
torch.cuda.is_available() False


In [8]:
# Outlier detection specification
if args.outlier_detection is not None:
    outlier_columns = ['rb_down', 'rb_up', 'down', 'up']
    outlier_kwargs = {"ElBorn": (10, 90), "LesCorts": (10, 90), "PobleSec": (5, 95)}
    args.outlier_columns = outlier_columns
    args.outlier_kwargs = outlier_kwargs

In [9]:
def seed_all():
    # ensure reproducibility
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
seed_all()

### The preprocessing pipeline performed here for the base station specified in filter_bs argument
Preprocessing inlcudes:
1. NaNs Handling NANs处理
2. Outliers Handling 异常值处理
3. Scaling Data 数据缩放
4. Generating time lags 生成时间滞后
5. Generating and importing exogenous data as features (time, statistics) (if applied) 生成和导入外生数据

In [11]:
def make_preprocessing(filter_bs=None):
    """Preprocess a given .csv"""
    # read data
    df = read_data(args.data_path, filter_data=filter_bs)
    # handle nans
    df = handle_nans(train_data=df, constant=args.nan_constant,
                     identifier=args.identifier)
    # split to train/validation
    train_data, val_data = to_train_val(df)
    
    # handle outliers (if specified)
    if args.outlier_detection is not None:
        train_data = handle_outliers(df=train_data, columns=args.outlier_columns,
                                     identifier=args.identifier, kwargs=args.outlier_kwargs)
    
    # get X and y
    X_train, X_val, y_train, y_val = to_Xy(train_data=train_data, val_data=val_data,
                                          targets=args.targets)
    
    # scale X
    X_train, X_val, x_scaler = scale_features(train_data=X_train, val_data=X_val,
                                             scaler=args.x_scaler, identifier=args.identifier)
    # scale y
    y_train, y_val, y_scaler = scale_features(train_data=y_train, val_data=y_val,
                                             scaler=args.y_scaler, identifier=args.identifier)
    
    # generate time lags
    X_train = generate_time_lags(X_train, args.num_lags)
    X_val = generate_time_lags(X_val, args.num_lags)
    y_train = generate_time_lags(y_train, args.num_lags, is_y=True)
    y_val = generate_time_lags(y_val, args.num_lags, is_y=True)
    
    # get datetime features as exogenous data
    date_time_df_train = time_to_feature(
        X_train, args.use_time_features, identifier=args.identifier
    )
    date_time_df_val = time_to_feature(
        X_val, args.use_time_features, identifier=args.identifier
    )
    
    # get statistics as exogenous data
    stats_df_train = assign_statistics(X_train, args.assign_stats, args.num_lags,
                                       targets=args.targets, identifier=args.identifier)
    stats_df_val = assign_statistics(X_val, args.assign_stats, args.num_lags, 
                                       targets=args.targets, identifier=args.identifier)
    
    # concat the exogenous features (if any) to a single dataframe
    if date_time_df_train is not None or stats_df_train is not None:
        exogenous_data_train = torch.pd.concat([date_time_df_train, stats_df_train], axis=1)
        # remove duplicate columns (if any)
        exogenous_data_train = exogenous_data_train.loc[:, ~exogenous_data_train.columns.duplicated()].copy()
        assert len(exogenous_data_train) == len(X_train) == len(y_train)
    else:
        exogenous_data_train = None
    if date_time_df_val is not None or stats_df_val is not None:
        exogenous_data_val = torch.pd.concat([date_time_df_val, stats_df_val], axis=1)
        exogenous_data_val = exogenous_data_val.loc[:, ~exogenous_data_val.columns.duplicated()].copy()
        assert len(exogenous_data_val) == len(X_val) == len(y_val)
    else:
        exogenous_data_val = None
        
    return X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler

In [15]:
# here exogenous_data_train and val are None.
X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler = make_preprocessing(
    filter_bs="LesCorts"
)

INFO logger 2024-06-03 23:59:48,001 | data_utils.py:28 | Reading LesCorts's data...
INFO logger 2024-06-03 23:59:48,021 | data_utils.py:565 | 	Total number of samples:  6892
INFO logger 2024-06-03 23:59:48,021 | data_utils.py:566 | 	Number of samples for training: 5514
INFO logger 2024-06-03 23:59:48,022 | data_utils.py:567 | 	Number of samples for validation:  1378


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
X_train.head()
#time：时间戳，表示数据记录的具体时间
#down：下行流量，单位可能是字节
#up：上行流量，单位可能是字节
#rnti_count：RNTI（Radio Network Temporary Identifier）计数，表示在指定时间内活跃的RNTI数量。
#mcs_down：下行调制和编码方案（MCS，Modulation and Coding Scheme）的平均值，表示下行链路的调制和编码效率。
#mcs_down_var：下行MCS的方差，表示下行链路调制和编码效率的变化程度
#mcs_up：上行调制和编码方案（MCS，Modulation and Coding Scheme）的平均值，表示上行链路的调制和编码效率。
#mcs_up_var：上行MCS的方差，表示上行链路调制和编码效率的变化程度
#rb_down：下行资源块（RB，Resource Block）的平均值，表示在指定时间内下行链路使用的资源块的平均数量。
#rb_down_var：下行资源块的方差，表示下行链路资源块使用数量的变化程度
#rb_up：上行资源块（RB，Resource Block）的平均值，表示在指定时间内上行链路使用的资源块的平均数量
#rb_up_var：上行资源块的方差，表示上行链路资源块使用数量的变化程度

,rb_up_var_lag-10,rb_up_lag-10,rb_down_var_lag-10,rb_down_lag-10,mcs_up_var_lag-10,mcs_up_lag-10,mcs_down_var_lag-10,mcs_down_lag-10,rnti_count_lag-10,up_lag-10,...,rb_down_var_lag-1,rb_down_lag-1,mcs_up_var_lag-1,mcs_up_lag-1,mcs_down_var_lag-1,mcs_down_lag-1,rnti_count_lag-1,up_lag-1,down_lag-1,District
time,,,,,,,,,,,,,,,,,,,,,
2019-01-12 17:32:00,1.083682e-08,0.036054,1.961025e-08,0.109670,0.493544,0.431702,0.459601,0.160269,0.210509,0.017705,...,1.974816e-08,0.114193,0.572030,0.400005,0.460896,0.164719,0.212757,0.018888,0.131219,LesCorts
2019-01-12 17:34:00,1.061491e-08,0.030426,1.644211e-08,0.098938,0.527483,0.476772,0.392094,0.131085,0.200300,0.018282,...,1.791601e-08,0.105544,0.576015,0.445876,0.427184,0.145817,0.192432,0.014424,0.113276,LesCorts
2019-01-12 17:36:00,9.109702e-09,0.038901,1.761309e-08,0.106045,0.501063,0.471773,0.453949,0.152725,0.221468,0.019501,...,1.766410e-08,0.101075,0.640010,0.448971,0.387407,0.130386,0.173137,0.011194,0.101414,LesCorts
2019-01-12 17:38:00,5.813802e-09,0.026050,1.668480e-08,0.098968,0.499865,0.450949,0.382516,0.129036,0.183768,0.012594,...,1.963081e-08,0.114777,0.534812,0.414812,0.476978,0.171880,0.205545,0.023585,0.141225,LesCorts
2019-01-12 17:40:00,7.246522e-09,0.028981,1.721707e-08,0.102669,0.535319,0.429488,0.412994,0.139577,0.191964,0.013056,...,1.895823e-08,0.107131,0.547422,0.456544,0.448805,0.157300,0.181567,0.022127,0.125069,LesCorts


In [ ]:
#解释
X_val.head()


,rb_up_var_lag-10,rb_up_lag-10,rb_down_var_lag-10,rb_down_lag-10,mcs_up_var_lag-10,mcs_up_lag-10,mcs_down_var_lag-10,mcs_down_lag-10,rnti_count_lag-10,up_lag-10,...,rb_down_var_lag-1,rb_down_lag-1,mcs_up_var_lag-1,mcs_up_lag-1,mcs_down_var_lag-1,mcs_down_lag-1,rnti_count_lag-1,up_lag-1,down_lag-1,District
time,,,,,,,,,,,,,,,,,,,,,
2019-01-20 09:20:00,4.728611e-09,0.000378,4.323541e-08,0.058302,0.401278,0.501344,0.919491,0.935376,0.123355,0.000206,...,4.402667e-08,0.072691,0.111839,0.444416,0.885758,0.931423,0.154451,0.000336,0.180818,LesCorts
2019-01-20 09:22:00,9.995833e-10,0.000330,4.508320e-08,0.056214,0.159144,0.393817,0.841731,0.960886,0.118906,0.000160,...,4.474956e-08,0.109179,0.275191,0.471308,0.877144,0.933492,0.222077,0.000623,0.264679,LesCorts
2019-01-20 09:24:00,1.217479e-08,0.000859,4.771216e-08,0.097153,0.301473,0.467948,0.869250,0.931095,0.202079,0.000558,...,4.620672e-08,0.140548,0.407761,0.487723,0.900534,0.947047,0.287079,0.000887,0.343279,LesCorts
2019-01-20 09:26:00,9.579305e-09,0.000850,4.462743e-08,0.100799,0.333194,0.519904,0.868349,0.948227,0.202641,0.000568,...,4.512395e-08,0.098633,0.307474,0.474030,0.852834,0.955406,0.203906,0.000384,0.241753,LesCorts
2019-01-20 09:28:00,2.505653e-08,0.001541,4.706725e-08,0.135352,0.406647,0.504277,0.900401,0.945294,0.270641,0.001207,...,4.390479e-08,0.076930,0.214200,0.541009,0.900236,0.940772,0.160727,0.000374,0.191878,LesCorts


In [ ]:
y_train.head()

,rnti_count,rb_down,rb_up,down,up,District
time,,,,,,
2019-01-12 17:32:00,0.192432,0.105544,0.033785,0.113276,0.014424,LesCorts
2019-01-12 17:34:00,0.173137,0.101075,0.025216,0.101414,0.011194,LesCorts
2019-01-12 17:36:00,0.205545,0.114777,0.060088,0.141225,0.023585,LesCorts
2019-01-12 17:38:00,0.181567,0.107131,0.042592,0.125069,0.022127,LesCorts
2019-01-12 17:40:00,0.175900,0.101726,0.023463,0.107920,0.011213,LesCorts


In [ ]:
x_scaler, y_scaler

(MinMaxScaler(), MinMaxScaler())

### Postprocessing Stage

In this stage we transform data in a way that can be fed into ML algorithms.

In [ ]:
def make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler):
    """Make data ready to be fed into ml algorithms"""
    # if there are more than one specified areas, get the data per area
    if X_train[args.identifier].nunique() != 1:
        area_X_train, area_X_val, area_y_train, area_y_val = get_data_by_area(X_train, X_val,
                                                                              y_train, y_val, 
                                                                              identifier=args.identifier)
    else:
        area_X_train, area_X_val, area_y_train, area_y_val = None, None, None, None

    # Get the exogenous data per area.
    if exogenous_data_train is not None:
        exogenous_data_train, exogenous_data_val = get_exogenous_data_by_area(exogenous_data_train,
                                                                              exogenous_data_val)
    # transform to np
    if area_X_train is not None:
        for area in area_X_train:
            tmp_X_train, tmp_y_train, tmp_X_val, tmp_y_val = remove_identifiers(
                area_X_train[area], area_y_train[area], area_X_val[area], area_y_val[area])
            tmp_X_train, tmp_y_train = tmp_X_train.to_numpy(), tmp_y_train.to_numpy()
            tmp_X_val, tmp_y_val = tmp_X_val.to_numpy(), tmp_y_val.to_numpy()
            area_X_train[area] = tmp_X_train
            area_X_val[area] = tmp_X_val
            area_y_train[area] = tmp_y_train
            area_y_val[area] = tmp_y_val
    
    if exogenous_data_train is not None:
        for area in exogenous_data_train:
            exogenous_data_train[area] = exogenous_data_train[area].to_numpy()
            exogenous_data_val[area] = exogenous_data_val[area].to_numpy()
    
    # remove identifiers from features, targets
    X_train, y_train, X_val, y_val = remove_identifiers(X_train, y_train, X_val, y_val)
    assert len(X_train.columns) == len(X_val.columns)
    
    num_features = len(X_train.columns) // args.num_lags
    
    # to timeseries representation
    X_train = to_timeseries_rep(X_train.to_numpy(), num_lags=args.num_lags,
                                            num_features=num_features)
    X_val = to_timeseries_rep(X_val.to_numpy(), num_lags=args.num_lags,
                                          num_features=num_features)
    
    if area_X_train is not None:
        area_X_train = to_timeseries_rep(area_X_train, num_lags=args.num_lags,
                                                     num_features=num_features)
        area_X_val = to_timeseries_rep(area_X_val, num_lags=args.num_lags,
                                                   num_features=num_features)
    
    # transform targets to numpy
    y_train, y_val = y_train.to_numpy(), y_val.to_numpy()
    
    # centralized (all) learning specific
    if not args.filter_bs and exogenous_data_train is not None:
        exogenous_data_train_combined, exogenous_data_val_combined = [], []
        for area in exogenous_data_train:
            exogenous_data_train_combined.extend(exogenous_data_train[area])
            exogenous_data_val_combined.extend(exogenous_data_val[area])
        exogenous_data_train_combined = np.stack(exogenous_data_train_combined)
        exogenous_data_val_combined = np.stack(exogenous_data_val_combined)
        exogenous_data_train["all"] = exogenous_data_train_combined
        exogenous_data_val["all"] = exogenous_data_val_combined
    return X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val

In [ ]:
X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val = make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler)

In [14]:
X_train[:2]

NameError: name 'X_train' is not defined

In [ ]:
y_train[:2]

array([[0.19243199, 0.1055439 , 0.03378513, 0.11327641, 0.01442373],
       [0.17313728, 0.10107498, 0.02521631, 0.10141373, 0.01119418]],
      dtype=float32)

In [ ]:
len(X_train), len(X_val)

(5504, 1368)

### Define the input dimensions for the model architecture

In [ ]:
def get_input_dims(X_train, exogenous_data_train):
    if args.model_name == "mlp":
        input_dim = X_train.shape[1] * X_train.shape[2]
    else:
        input_dim = X_train.shape[2]
    
    if exogenous_data_train is not None:
        if len(exogenous_data_train) == 1:
            cid = next(iter(exogenous_data_train.keys()))
            exogenous_dim = exogenous_data_train[cid].shape[1]
        else:
            exogenous_dim = exogenous_data_train["all"].shape[1]
    else:
        exogenous_dim = 0
    
    return input_dim, exogenous_dim

### Initialize the model for training

In [ ]:
def get_model(model: str,
              input_dim: int,
              out_dim: int,
              lags: int = 10,
              exogenous_dim: int = 0,
              seed=0):
    if model == "mlp":
        model = MLP(input_dim=input_dim, layer_units=[256, 128, 64], num_outputs=out_dim)
    elif model == "rnn":
        model = RNN(input_dim=input_dim, rnn_hidden_size=128, num_rnn_layers=1, rnn_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "lstm":
        model = LSTM(input_dim=input_dim, lstm_hidden_size=128, num_lstm_layers=1, lstm_dropout=0.0,
                     layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "gru":
        model = GRU(input_dim=input_dim, gru_hidden_size=128, num_gru_layers=1, gru_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "cnn":
        model = CNN(num_features=input_dim, lags=lags, exogenous_dim=exogenous_dim, out_dim=out_dim)
    elif model == "da_encoder_decoder":
        model = DualAttentionAutoEncoder(input_dim=input_dim, architecture="lstm", matrix_rep=True)
    else:
        raise NotImplementedError("Specified model is not implemented. Plese define your own model or choose one from ['mlp', 'rnn', 'lstm', 'gru', 'cnn', 'da_encoder_decoder']")
    return model

In [ ]:
# define the model
args.model_name = "mlp"

input_dim, exogenous_dim = get_input_dims(X_train, exogenous_data_train)

print(input_dim, exogenous_dim)

model = get_model(model=args.model_name,
                  input_dim=input_dim,
                  out_dim=y_train.shape[1],
                  lags=args.num_lags,
                  exogenous_dim=exogenous_dim,
                  seed=args.seed)

110 0


In [ ]:
model

MLP(
  (MLP_layers): Sequential(
    (0): Linear(in_features=110, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=5, bias=True)
  )
)

### The fit function used to train the model specified above

In [ ]:
def fit(model, X_train, y_train, X_val, y_val, 
        exogenous_data_train=None, exogenous_data_val=None, 
        idxs=[8, 3, 1, 10, 9], # the indices of our targets in X
        log_per=1):
    
    # get exogenous data (if any)
    if exogenous_data_train is not None and len(exogenous_data_train) > 1:
        exogenous_data_train = exogenous_data_train["all"]
        exogenous_data_val = exogenous_data_val["all"]
    elif exogenous_data_train is not None and len(exogenous_data_train) == 1:
        cid = next(iter(exogenous_data_train.keys()))
        exogenous_data_train = exogenous_data_train[cid]
        exogenous_data_val = exogenous_data_val[cid]
    else:
        exogenous_data_train = None
        exogenous_data_val = None
    num_features = len(X_train[0][0])
    
    # to torch loader
    train_loader = to_torch_dataset(X_train, y_train,
                                    num_lags=args.num_lags,
                                    num_features=num_features,
                                    exogenous_data=exogenous_data_train,
                                    indices=idxs,
                                    batch_size=args.batch_size, 
                                    shuffle=False)
    val_loader = to_torch_dataset(X_val, y_val, 
                                  num_lags=args.num_lags,
                                  num_features=num_features,
                                  exogenous_data=exogenous_data_val,
                                  indices=idxs,
                                  batch_size=args.batch_size,
                                  shuffle=False)
    
    # train the model
    model = train(model, 
                  train_loader, val_loader,
                  epochs=args.epochs,
                  optimizer=args.optimizer, lr=args.lr,
                  criterion=args.criterion,
                  early_stopping=args.early_stopping,
                  patience=args.patience,
                  plot_history=args.plot_history, 
                  device=device, log_per=log_per)
    
    return model

In [13]:
trained_model = fit(model, X_train, y_train, X_val, y_val)

NameError: name 'fit' is not defined